In [1]:
import pandas as pd
from tqdm import tqdm
# post=pd.read_csv(f"concat_post.csv", encoding='utf_8_sig' )
floor=11
post=pd.read_csv(f"user_L{floor}_post_classify.csv", encoding='utf_8_sig')
# post=pd.read_csv(f"user_L{floor}_post_filter_temp.csv", encoding='utf_8_sig')
post=post[post['forum_category']=='手机']
print(post.shape[0])
# post['post_tag']=None
if 'post_tag' in post.columns:
    print('exist')
else:
    post['post_tag']=None

15623
exist


In [2]:
import requests
import time
access_token_list=[]
api_key='2VyIDSSm72ikpdNvHEFuSZYE'
client_secret='EuqOAwzQbC4oncTKDn9mcIMTRTavYeSi'
# client_id 为官网获取的AK， client_secret 为官网获取的SK
host = f'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id={api_key}&client_secret={client_secret}'
response = requests.get(host)
if response:
    print(response.json())
access_token=eval(response.text)['access_token']
access_token_list.append(access_token)

api_key='PGALriW1axDoujwcN0N2LCiD'
client_secret='x9d7s0Iat2DQsFbmOmkz8c4ezkot4VHm'
host = f'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id={api_key}&client_secret={client_secret}'
response = requests.get(host)
if response:
    print(response.json())
access_token_1=eval(response.text)['access_token']
access_token_list.append(access_token_1)

{'refresh_token': '25.2946a91295a8e7cbc61ccea0cde7ba36.315360000.1916965502.282335-18606470', 'expires_in': 2592000, 'session_key': '9mzdDZW60L8MF2yKUmeSLStNeIU5pQ/pesDN3ynXwcQxqnIQKzqiXdTeZzmhSOkCq0jgpwnJulhd9aVqNqfAO1aEVIz8LQ==', 'access_token': '24.6ca705df1c9afe8f50ac3111196c5da1.2592000.1604197502.282335-18606470', 'scope': 'brain_nlp_address public nlp_simnet nlp_wordemb nlp_comtag nlp_dnnlm_cn brain_nlp_lexer brain_all_scope brain_nlp_comment_tag brain_nlp_dnnlm_cn brain_nlp_word_emb_vec brain_nlp_word_emb_sim brain_nlp_sentiment_classify brain_nlp_simnet brain_nlp_depparser brain_nlp_wordembedding brain_nlp_dnnlm_cn_legacy brain_nlp_simnet_legacy brain_nlp_comment_tag_legacy brain_nlp_lexer_custom brain_nlp_keyword brain_nlp_topic brain_nlp_ecnet brain_nlp_emotion brain_nlp_comment_tag_custom brain_nlp_news_summary brain_nlp_sentiment_classify_custom wise_adapt lebo_resource_base lightservice_public hetu_basic lightcms_map_poi kaidian_kaidian ApsMisTest_Test权限 vis-classify_flow

In [3]:
import pandas as pd
from tqdm import tqdm
post['post_tag'].fillna('{}',inplace=True)
post['post_tag']=post['post_tag'].apply(lambda x:eval(x))

# 再次观点抽取

In [14]:
# host=f'https://aip.baidubce.com/rpc/2.0/nlp/v2/comment_tag?access_token={access_token}'
# # for index,row in tqdm(post.iterrows()):
# for index,row in tqdm(post.iterrows()):
#     try:
#         if row['post_tag'].get('error_code'):
#             content = {'text': row['post_title'], 'type': 13} # 3c
#             result = requests.post(url=host,json=content)
#             post.at[index,'post_tag']=result.text
#             time.sleep(0.4)
#     except BaseException as e:
#         pass
    # break
for index,row in tqdm(post.iterrows()):
    try:
        if row['post_tag'].get('error_code'):
            content = {'text': row['post_title'], 'type': 13}
            host=f'https://aip.baidubce.com/rpc/2.0/nlp/v2/comment_tag?access_token={access_token_list[index%2]}'
            result = requests.post(url=host,json=content)
            post.at[index,'post_tag']=result.text
            time.sleep(0.25)
    except BaseException:
        pass

15623it [00:01, 7992.25it/s] 


In [17]:
import time
# host='https://aip.baidubce.com/rpc/2.0/nlp/v2/comment_tag'
# host=f'https://aip.baidubce.com/rpc/2.0/nlp/v2/comment_tag?access_token={access_token}'
# battery_post['post_tag']=None
for index,row in tqdm(post.iterrows()):
    try:
        if not row['post_tag'].get('log_id'):
            content = {'text': row['post_title'], 'type': 13}
            host=f'https://aip.baidubce.com/rpc/2.0/nlp/v2/comment_tag?access_token={access_token_list[index%2]}'
            result = requests.post(url=host,json=content)
            post.at[index,'post_tag']=result.text
            time.sleep(0.25)
    except BaseException:
        pass
    # break
temporary_pd=post

15623it [00:01, 9018.30it/s] 


In [5]:
# post.to_csv(f"user_L{floor}_post_filter_temp.csv", encoding='utf_8_sig', index=False)

In [ ]:
def extract_opinion(df):
    count=0
    df['prop_adj_list']=None
    df['sentiment_list']=None
    try:
        df['post_tag']=df['post_tag'].apply(lambda x:eval(str(x)))
    except BaseException as e:
        print(df['post_tag'].dtype)
        print(df['post_tag'].isna().sum())
        print(e)
    for _index,_row in tqdm(df.iterrows()):
        tmp=[]
        tmp_sentiment=[]
        try:
            for cell in _row['post_tag']['items']:
                tmp.append(cell['prop']+cell['adj'])
                tmp_sentiment.append(cell['sentiment'])
                count+=1
            df.at[_index,'prop_adj_list']=tmp
            df.at[_index,'sentiment_list']=tmp_sentiment
        except BaseException as e:
            print(e,_row)
            # print(e,_row['post_tag']['error_code'])
    print(count,'个观点')

    return df

tmp_pd=extract_opinion(post)

In [20]:
post=post[~post['sentiment_list'].isna()]
post['sentiment_list']=post['sentiment_list'].astype('str')
post=(post[post['sentiment_list'].str.len()>2])

15623it [00:01, 8012.57it/s]


'items' author_user_name                                               宝宝乐752
post_title                                                          🐮
url                  /p/5999615046?lp=home_main_thread_pb&mo_device=1
bar_name                                                          手机吧
forum_category                                                     手机
post_tag            {'log_id': 1817883952890751554, 'error_code': ...
prop_adj_list                                                    None
sentiment_list                                                   None
Name: 6155, dtype: object
'items' 282134
1825 个观点


In [21]:
post=post[~post['sentiment_list'].isna()]
post['sentiment_list']=post['sentiment_list'].astype('str')
post=(post[post['sentiment_list'].str.len()>2])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
import re
post['post_id']=None
for index,row in tqdm(post.iterrows()):
    post.at[index,'post_id']= re.findall('/p/([0-9]+)\?',row['url'])[0]

1693it [00:00, 9077.81it/s]


In [23]:
post.to_csv(f"user_L{floor}_post_filter.csv", encoding='utf_8_sig', index=False)

